In [2]:
from bs4 import BeautifulSoup
from requests import get
import numpy as np
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import itertools
import json
from shapely.geometry import Point
import matplotlib.pyplot as plt
# Standard plotly imports
import chart_studio.plotly as py
# Using plotly + cufflinks in offline mode
import cufflinks
import plotly.express as px
cufflinks.go_offline(connected=True)
import re
import time

In [3]:
def GetHtmlSoup(url, headers=()):
    try:
        response = get(url, headers=headers)
    except Exception as e:
        raise Exception(e)
        
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    return html_soup

In [4]:
def GetRetailContainer(url, name, attrs, **kwargs):
    html_soup = GetHtmlSoup(url)
    
    retail_container = html_soup.find_all(name, attrs, **kwargs)
    
    return retail_container

In [5]:
url = 'https://www.spacelist.ca/listings/b/dpz3q4u51j/dpxxegdyr8/retail-other/for-lease'
html_soup = GetHtmlSoup(url)

In [6]:
map_container = html_soup.find('div', {'id': 'cluster-map'})['data-data']
map_dict_list = json.loads(map_container)['features']
coord_dict = {f['properties']['id']: Point(f['geometry']['coordinates']) for f in map_dict_list}

In [40]:
n_pages = 70
n_ignored = 0

business_dict = {'Id': [], 'Address': [], 'Area sf': [], 'Rent/mo': [], 'Type': [], 'geometry': []}
for page in range(1, n_pages + 1):
    url = 'https://www.spacelist.ca/listings/on/toronto/office-retail-industrial-medical-multifamily-other/for-lease/page/' + str(page)
    
    retail_container = GetRetailContainer(url, 'div', {'class': 'listing-result cell shrink'})
    
    if retail_container != []:
        page_location = 0
        for retail in retail_container:
            page_location += 1
            id_value = retail.find('a', {'class': 'add-to-shortlist'})['data-listing-id']
            
            try:
                address_value = retail.find(
                    'div',
                    {'class': 'cell auto small-font flat-line-height-taller right-cell-padding'})
                address_value = address_value.find('div', {'class': 'dark-font'}).get('title')
            except:
                n_ignored += 1
                continue
            
            try:
                area_value = float(
                    re.findall(
                        '\d+(?:,\d+)?', 
                        retail.find('div', {'class': 'default-font dark-font flat-line-height'}).text
                    )[0].replace(',', '')
                )
            except:
                n_ignored += 1
                continue
                
            rent_text = retail.find('div', {'class': 'rent_per_month'})
            
            if rent_text is not None:
                rent_text = rent_text.text                
            else:
                sub_url = 'https://www.spacelist.ca/listings/' + id_value + '/on/toronto/for-lease/retail'
                
                sub_html_soup = GetHtmlSoup(sub_url)
                retail_container = sub_html_soup.find_all('div', {'class': 'cell small-12 large-6'})
                rent_text = retail_container[0].find('span', {'class': 'bold-font'}).text

            try:
                rent_value = float(re.search(r'\$(.*?)\/', rent_text).group(1).replace(',', ''))
            except:
                n_ignored += 1
                continue
                
            if 'sf/yr' in rent_text:
                rent_value = (rent_value * area_value) / 12

            type_value = retail.find('div', {'class': 'cell auto about'}).contents[1].text
            
            try:
                coord_value = coord_dict[int(id_value)]
            except:
                coord_value = np.nan
            
            business_dict['Id'].append(id_value)
            business_dict['Address'].append(address_value)
            business_dict['Area sf'].append(area_value)
            business_dict['Rent/mo'].append(rent_value)
            business_dict['Type'].append(type_value)
            business_dict['geometry'].append(coord_value)
    else:
        break
        
print('Number of spaces scraped:', len(business_dict['Id']), '\n')
print('Number of spaces ignored:', n_ignored)

Number of spaces scraped: 660 

Number of spaces ignored: 740


In [41]:
retail_sl_df = pd.DataFrame(business_dict)
retail_sl_df.head()

,Id,Address,Area sf,Rent/mo,Type,geometry
0,191578,301 - 32 Britain St,165.0,2900.0,Office,NaN
1,278433,404 - 32 Britain St,165.0,3500.0,Office,NaN
2,184289,403 - 32 Britain St,260.0,6000.0,Office,NaN
3,319350,401 - 32 Britain St,350.0,6600.0,Office,NaN
4,284426,For Rent - 26 Duncan St,1200.0,8300.0,Office,NaN


In [42]:
retail_sl_df.shape

(660, 6)

In [48]:
n_unk_coord = retail_sl_df.loc[retail_sl_df['geometry'].isnull()].shape[0]
print('Number of unknown coordinates:', n_unk_coord)

Number of unknown coordinates: 660


In [49]:
retail_sl_df['Address'] = retail_sl_df.loc[:, ('Address')].apply(
    lambda s: s.replace(
        'For Rent - ', '').replace(
        'For Rent  - ', '').replace(
        'COWORKING DEDICATED - ', '').replace(
        ' COWORKING DEDICATED - ', '').replace(
        'COWORKING NOT RESERVED - ', '').replace(
        ' COWORKING NOT RESERVED - ', ''))
retail_sl_df['Main Address'] = retail_sl_df.loc[:, ('Address')].apply(
    lambda s: s.split('-')[-1].lstrip() if ('-' in s and 'HWY' not in s) else ('-'.join(s.split('-')[-2:]).lstrip() if 'HWY' in s else s))

In [53]:
na_address = np.unique(retail_sl_df.loc[retail_sl_df['geometry'].isnull()]['Main Address'])

coord_dict = {'Main Address': [], 'geometry': []}
geolocator = Nominatim(user_agent='web_scraper')
for addr in na_address:
    try:
        location = geolocator.geocode(addr + ' Toronto', timeout=10)
    except Exception as e:
        print(addr, e)
        continue
    try:
        addr_point = Point(location.longitude, location.latitude)
    except Exception as e:
        print(addr, e)
        continue
    coord_dict['geometry'].append(addr_point)
    coord_dict['Main Address'].append(addr)

388 Carlaw Ave, Unit W20 'NoneType' object has no attribute 'longitude'


In [54]:
geo_addr_gdf = gpd.GeoDataFrame(pd.DataFrame.from_dict(coord_dict))

retail_sl_df = retail_sl_df.merge(geo_addr_gdf, how='left', on='Main Address')
retail_sl_df.loc[retail_sl_df['geometry_x'].isnull(), 'geometry_x'] = retail_sl_df.loc[
    ~retail_sl_df['geometry_y'].isnull(), 'geometry_y']
retail_sl_df.drop('geometry_y', axis=1, inplace=True)
retail_sl_df.rename(columns={'geometry_x': 'geometry'}, inplace=True)

n_unk_coord = retail_sl_df.loc[retail_sl_df['geometry'].isnull()].shape[0]
print('Number of unknown coordinates:', n_unk_coord)

Number of unknown coordinates: 1


In [55]:
n_pages = 15
n_ignored = 0

business_dict_p2h = {'Id': [], 'Address': [], 'Area sf': [], 'Rent/mo': [], 'Type': [], 'geometry': []}
for page in range(1, n_pages + 1):
    url = 'https://www.point2homes.com/CA/Commercial-Properties-For-Lease.html?location=Toronto%2C+ON&PriceMin=0&PriceMax=1%2C000%2C000&search_mode=location&geopicker_output=853998%2C854002%2C854003%2C854008%2C854009%2C854010%2C854011%2C855493%2C855558%2C855562%2C855563&geopicker_type=nhood&geopicker_output_full=853998%7CEast+York%7COntario%2C854002%7CUptown+Toronto%7COntario%2C854003%7CYork%7COntario%2C854008%7CWest+End+Toronto%7COntario%2C854009%7CMidtown+Toronto%7COntario%2C854010%7CDowntown+Toronto%7COntario%2C854011%7CEast+End+Toronto%7COntario%2C855493%7CCaribou+Park%7CNorth+York%2C855558%7CGlen+Park%7CNorth+York%2C855562%7CLedbury+Park%7CNorth+York%2C855563%7CLawrence+Manor%7CNorth+York&page={}&SelectedView=listings&LocationGeoId=783094&viewport=-79.46408975087829%2C43.659463577335885%2C-79.35886132680602%2C43.76728718084169&zoom=13&location_changed=&ajax=1'.format(page)
    retail_container_p2h = GetRetailContainer(url, 'div', {'class': 'item-cnt clearfix'})

    if retail_container_p2h != []:
        for retail in retail_container_p2h:
            id_value = retail['id']
            
            # the lot size can be in acres, square feet, or it isn't listed
            # if it isn't listed then ignore the retail and continue the loop
            if retail.find('li', {'data-label': 'Lot Size'}) is None:
                if retail.find('li', {'data-label': 'Sqft'}) is None:
                    n_ignored += 1
                    continue
                else:
                    area_value = float(
                        re.findall(
                            '\d+(?:,\d+)?', 
                            retail.find('li', {'data-label': 'Sqft'}).text
                        )[0].replace(',', '')
                    )
            else:
                # convert acre to square feet
                area_value = float(
                    re.findall(
                        '\d+(?:.\d+)?',
                        retail.find('li', {'data-label': 'Lot Size'}).text
                    )[0]
                ) * 43560
            
            # if the rent is not given as a per month value, ignore it and continue the loop
            # the reason being that sometimes multiple spaces are listed with different rents in a
            # single listing, and the individual area and rent of each space is either not listed
            # or it's listed in paragraph form in a description field
            try:
                rent_value_month = float(
                    re.findall(
                        '\d+\,\d+', 
                        retail.find('div', {'class': 'price has-rental-term'})['data-price']
                    )[0].replace(',', '')
                )
            except:
                n_ignored += 1
                continue
            
            # it makes no sense that the rent would be $0 so ignore these ones
            if rent_value_month == 0:
                n_ignored += 1
                continue
            
            type_value = retail.find('li', {'class': 'property-type ic-proptype'}).text
            type_value = ''.join(e for e in type_value if e.isalnum())

            input_list = retail.find_all('input')
            address_value = input_list[0]['value'].split(',')[0]
            lat_value = float(input_list[1]['value'])
            lng_value = float(input_list[2]['value'])
            coord_value = Point(lng_value, lat_value)
            
            business_dict_p2h['Id'].append(id_value)
            business_dict_p2h['Address'].append(address_value)
            business_dict_p2h['Area sf'].append(area_value)
            business_dict_p2h['Rent/mo'].append(rent_value_month)
            business_dict_p2h['Type'].append(type_value)
            business_dict_p2h['geometry'].append(coord_value)
    else:
        break
        
print('Number of spaces scraped:', len(business_dict_p2h['Id']), '\n')
print('Number of spaces ignored:', n_ignored)

Number of spaces scraped: 171 

Number of spaces ignored: 136


In [56]:
retail_p2h_gdf = gpd.GeoDataFrame(pd.DataFrame(business_dict_p2h))
retail_p2h_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry
0,l_94697677,507 Queen St W,2530.0,8000.0,Commercial,POINT (-79.39868 43.64806)
1,l_94697684,454 Queen St W,1100.0,8000.0,Commercial,POINT (-79.39945 43.64834)
2,l_92496157,634 Queen St W,1100.0,5000.0,Commercial,POINT (-79.40593 43.64697)
3,l_94697679,693 Queen St W,2000.0,10834.0,Commercial,POINT (-79.40527 43.64680)
4,l_92817532,452 Queen St West,1100.0,8000.0,Commercial,POINT (-79.39940 43.64831)


In [57]:
retail_p2h_gdf.shape

(171, 6)

In [58]:
retail_gdf = gpd.GeoDataFrame(pd.concat([retail_sl_df, retail_p2h_gdf], ignore_index=True))
retail_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry,Main Address
0,191578,301 - 32 Britain St,165.0,2900.0,Office,POINT (-79.37050 43.65389),32 Britain St
1,278433,404 - 32 Britain St,165.0,3500.0,Office,POINT (-79.37050 43.65389),32 Britain St
2,184289,403 - 32 Britain St,260.0,6000.0,Office,POINT (-79.37050 43.65389),32 Britain St
3,319350,401 - 32 Britain St,350.0,6600.0,Office,POINT (-79.37050 43.65389),32 Britain St
4,284426,26 Duncan St,1200.0,8300.0,Office,POINT (-79.38911 43.64859),26 Duncan St


In [60]:
retail_gdf.shape

(831, 7)

In [61]:
# create columns for unique address IDs regardless of the source and format of the address
retail_gdf['Address ID'] = retail_gdf['Address'].apply(lambda s: ''.join(sorted(str.upper(s.replace('-', '').replace(' ', '')))))
retail_gdf['ID length'] = retail_gdf['Id'].apply(len)

In [62]:
# deal with duplicates
print(retail_gdf.shape)
retail_gdf.drop_duplicates(subset=['Address ID', 'Area sf', 'Rent/mo'], inplace=True)
print(retail_gdf.shape)
retail_gdf.drop_duplicates(subset=['Address ID', 'ID length'], inplace=True)
print(retail_gdf.shape)
retail_gdf.drop_duplicates(subset=['Address ID'], inplace=True)
print(retail_gdf.shape)

(831, 9)
(816, 9)
(789, 9)
(788, 9)


In [63]:
retail_gdf.drop(['Address ID', 'ID length'], axis=1, inplace=True)
retail_gdf.reset_index(drop=True, inplace=True)
retail_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry,Main Address
0,191578,301 - 32 Britain St,165.0,2900.0,Office,POINT (-79.37050 43.65389),32 Britain St
1,278433,404 - 32 Britain St,165.0,3500.0,Office,POINT (-79.37050 43.65389),32 Britain St
2,184289,403 - 32 Britain St,260.0,6000.0,Office,POINT (-79.37050 43.65389),32 Britain St
3,319350,401 - 32 Britain St,350.0,6600.0,Office,POINT (-79.37050 43.65389),32 Britain St
4,284426,26 Duncan St,1200.0,8300.0,Office,POINT (-79.38911 43.64859),26 Duncan St


In [64]:
print(retail_gdf.shape)

(788, 7)


In [65]:
retail_gdf.to_file('rent_toronto.geojson', drive='GeoJSON', encoding='utf-8')